In [105]:
from typing import Tuple
import re
import json
from collections import defaultdict, OrderedDict

In [2]:
import numpy as np
import pandas as pd

## Read namuwiki dump

In [3]:
with open('../data/namuwiki.json') as f:
    context_ = json.load(f)

In [4]:
context = defaultdict(dict)
for doc in context_:
    context[doc['title']][doc['namespace']] = doc

## Parser

In [154]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('\|\|(.+?)\|\|(.+?)\|\|')

regex_tags = OrderedDict({
    'regex_del': (r'\1', re.compile('~~(.+?)~~')),
    'regex_dash': (r'\1', re.compile('--(.+?)--')),
    'regex_color': (r'\2', re.compile('\{\{\{\#(.+?) \'\'\'(.+?)\'\'\' \}\}\}')),
    'regex_link': (r'\1', re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')),
    'regex_attach': ('attach', re.compile('\[\[파일:(.+?)\]\]')),
    'regex_document': (r'\1', re.compile('\[\[(.[^:]+?)\]\]')),
    
    'regex_tag': ('', re.compile('\<(.+?)\>')),
    'regex_comment': ('', re.compile('\[\*(.+?)\]')),
})

In [155]:
def parse(text: str) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) -> Tuple[str, int]:
        return tag.subn(target, text)
    
    count = 0
    
    while True:
        for target_tag in regex_tags.values():
            text, count = _parse(text, *target_tag)
            if count:
                break
        if not count:
            break
            
    return text.strip()

## Get drama titles from channles

In [156]:
interested = ["JTBC 금토 드라마(2014~2017)", "JTBC 금토 드라마(2017~2020)", "JTBC 드라마", "JTBC 수목 드라마", "JTBC 월화 드라마(2011~2014)", "JTBC 월화 드라마(2017~2020)", "JTBC 주말 드라마", "KBS 수목 드라마(2001~2005)", "KBS 수목 드라마(2006~2010)", "KBS 수목 드라마(2011~2015)", "KBS 수목 드라마(2016~2020)", "KBS 월화 드라마(2001~2005)", "KBS 월화 드라마(2006~2010)", "KBS 월화 드라마(2011~2015)", "KBS 월화 드라마(2016~2020)", "KBS 학교 시리즈", "MBC 수목 미니시리즈(2006~2010)", "MBC 수목 미니시리즈(2011~2015)", "MBC 수목 미니시리즈(2016~2020)", "MBC 아침 드라마(2011~2015)", "MBC 아침 드라마(2016~2020)", "MBC 예능 드라마", "MBC 월화 미니시리즈(2006~2010)", "MBC 월화 미니시리즈(2016~2020)", "MBC 월화특별기획(2011~2015)", "MBC 일일 드라마(2016~2020)", "MBC 일일 연속극(2011~2015)", "MBC 주말 드라마(2011~2015)", "MBC 주말 드라마(2016~2020)", "MBC 주말 특별기획(2011~2015)", "MBC 주말 특별기획(2016~2020)", "MBC 하이킥 시리즈", "MBN 수목 드라마", "OCN 로맨스 드라마", "OCN 수목 오리지널", "OCN 오리지널 드라마(2010~2016)", "OCN 월화 오리지널", "OCN 토일 오리지널(2017~2020)", "SBS 금토 드라마(2019~현재)", "SBS 드라마 스페셜(1992~1995)", "SBS 드라마 스페셜(1996~2000)", "SBS 드라마 스페셜(2001~2005)", "SBS 드라마 스페셜(2006~2010)", "SBS 드라마 스페셜(2011~2015)", "SBS 드라마 스페셜(2016~2020)", "SBS 아침 연속극(2016~2020)", "SBS 월화 드라마(1991~1995)", "SBS 월화 드라마(1996~2000)", "SBS 월화 드라마(2001~2005)", "SBS 월화 드라마(2006~2010)", "SBS 월화 드라마(2011~2015)", "SBS 월화 드라마(2016~2020)", "TV CHOSUN 토일드라마", "tvN 금요 드라마(2007~2015)", "tvN 금토 드라마", "tvN 로맨스가 필요해 시리즈", "tvN 불금 시리즈(2017~)", "tvN 월화 드라마(2011~2015)", "tvN 월화 드라마(2016~2020)", "tvN 토일 드라마(2017~2020)"]

In [157]:
titles = set()

In [158]:
for inter in interested:
    document = context[inter]
    matches = regex_document.findall(document['1']['text'])
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        titles.add(name)

## Get metadata from document

In [159]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [177]:
data = defaultdict(dict)
notexists = []
notparser = []

In [178]:
for title in titles:
    try:
        document = context[title]['0']['text']                       
    except KeyError:
        notexists.append(title)
        
    matches = regex_table.findall(document)
    
    d = defaultdict(str)
    
    for key, value in matches:
        key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
        if key:
            key = key.strip()
            d[key] = f'{d[key]} {value}'
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [179]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

126 pages are not parsable
2 pages are not exists


## Create table from data

In [180]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        title, *tuple(map(parse, map(lambda c: values[c], table_columns)))
    ])))

In [181]:
df = pd.DataFrame(table)
df.columns = ['title', *table_columns]

In [182]:
print(df.shape)
df.head()

(735, 8)


,title,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자
0,강남 스캔들,2018년 11월 26일 ~ 2019년 5월 17일,123부작,"막장드라마, 가족드라마",SBS,SBS Plus 제작,박혜련,"신고은(가수), 임윤호, 서도영, 황보미(방송인), 방은희 外"
1,남자가 사랑할 때(SBS 드라마),2004년 9월 30일 ~ 2004년 11월 18일,16부작,,SBS,김종학프로덕션,김윤정,"고수(배우), 박정아, 박예진, 배수빈 外"
2,유령(드라마),2012년 5월 30일 ~ 2012년 8월 9일,20부작,"범죄, 스릴러",SBS,HB엔터테인먼트,김은희,"소지섭, 이연희(배우), 엄기준, 곽도원"
3,"하이바이, 마마!",,16부작,,대한민국 : tvN [br] 세계 : Netflix,"스튜디오 드래곤, 엠아이",권혜주,"김태희, 이규형(배우), 고보결 外"
4,패션 70's,2005년 5월 23일 ~ 2005년 8월 29일,28부작,,SBS,"김종학프로덕션, 세정]]",정성희,"이요원, 김민정(1982), 주진모(1974), 천정명 外"


In [183]:
df.to_csv('../results/namuwiki.csv', index=None)